In [1]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core import Workspace
ws = Workspace.from_config()
cluster_name = "udaciy-yum"
# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
### YOUR CODE HERE ###
compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
yum_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
yum_cluster.wait_for_completion(show_output=True)

InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [5]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
from azureml.core.experiment import Experiment
import os

# Specify parameter sampler
# ps = ### YOUR CODE HERE ###
ps = RandomParameterSampling( {
        "C": uniform(0.05, 0.1),
        "max_iter": choice(10, 20, 40, 60)
    }
)

# Specify a Policy
# policy = ### YOUR CODE HERE ###
policy = BanditPolicy(evaluation_interval = 5, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

In [6]:
# Create a ScriptRunConfig Object to specify the configuration details of your training job
# src = ### YOUR CODE HERE ###

# create or load an experiment
experiment = Experiment(ws, 'LRExperiment')
src = ScriptRunConfig(source_directory='.',
                        script='train.py',
                        compute_target=yum_cluster,
                        environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
# hyperdrive_config = ### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(run_config=src,
                                hyperparameter_sampling=ps,
                                policy=policy,
                                primary_metric_name='accuracy',
                                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                max_total_runs=20,
                                max_concurrent_runs=4)

In [7]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hd_run = experiment.submit(config=hyperdrive_config)
RunDetails(hd_run).show()


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [8]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###

best_run = hd_run.get_best_run_by_primary_metric()
joblib.dump(value= best_run, filename = './hd_best_model.joblib')

# # tag the runs 
# run1.add_properties("author":"yum")
# run1.tag("quality","best_run")
# list(experiment.get_runs(properties = {"author":"yum"}, tags = {"quality" : "best_run"}))
# best_model = experiment.get_runs(properties = {"author":"yum"}, tags = {"quality" : "best_run"})

['./hd_best_model.joblib']

In [9]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
url = ['https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv']
dataset = TabularDatasetFactory.from_delimited_files(url)
ds = dataset.to_pandas_dataframe() 
ds.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,...,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1,no
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,...,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1,no
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,...,4,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1,no
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,no


In [10]:
from train import clean_data
from sklearn.model_selection import train_test_split
import pandas as pd

# Use the clean_data function to clean your data.
x, y = clean_data(dataset)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2022)
df_train = pd.concat([x_train,y_train], axis=1)

In [11]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task= 'classification',
    primary_metric= 'accuracy',
    training_data=df_train,
    label_column_name='y',
    n_cross_validations=5)

In [12]:
# Submit your automl run

### YOUR CODE HERE ###
auto_run = experiment.submit(automl_config, show_output = True)

No run_configuration provided, running on local with default configuration
Running in the active local environment.


Experiment,Id,Type,Status,Details Page,Docs Page
LRExperiment,AutoML_504c039e-f2ec-4761-a5c6-bd934ee19f2e,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.


In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###

best_auto_run = auto_run.get_best_run_by_primary_metric()
joblib.dump(value= best_auto_run, filename = './auto_best_model.joblib')

# autorun.add_properties("author":"yum")
# autorun.tag("quality","best_auto_run")
# list(experiment.get_runs(properties = {"author":"yum"}, tags = {"quality" : "best_auto_run"}))
# best_auto_model = experiment.get_runs(properties = {"author":"yum"}, tags = {"quality" : "best_auto_run"})